##### TODO: 

I) version base
(partiel)- sauver prédictions sur dev (ou test); script

      
/home/muller/anaconda3/envs/allennlp/lib/python3.7/site-packages/allennlp/data/dataset_readers/semantic_dependency_parsing.py


améliorations: 
- encodeur tour -> bert
     deux solutions 
     1) bertpooler direct sur passthrough embeddings 
     2) indexer/tokenizer bert + seq2vec ?  (pb de stockage tokens...)
- preprocessing des chats
    - turn level feature extraction + intégration au modèle
        - (done) 1 trait (is_server)
        - generaliser
    - pair level feat. extr. + integration au modèle
        cf coref model
         - (done) avec 1 ou 2 traits (distance + test si target addresse = src speaker)
         - https://github.com/allenai/allennlp-models/blob/master/allennlp_models/coref/models/coref.py
         - https://github.com/allenai/allennlp/blob/master/allennlp/nn/util.py (for distance etc)
    - (done) alternative pour distance: embedding de la position avec sinusoides
    - ajouter embedding/encodage niveau caractères vue la nature du chat
    - utiliser les embeddings glove entrainés sur ubuntu fournis par kummerfeld 
- changer les instances
    - small sequence (10-15 tours), predire un seul lien 
         - (a) pour chaque source: contexte après, prédire lien chaque tour / src
         - (b) pour chaque target: contexte avant, prédire lien, chaque tour/target / garder le meilleur
- test softmax vs sigmoid in scores->probs
- better decoders
    - mst 
    - (done) best head for each target -> pas bcp mieux
- clean-up
     - metadata in instances: tokens, etc
     - model saving/loading from archive
     - separate prediction in dedicated script

data analysis:

  - eval in kummerfeld: is it for each "query" ? (query=turn with question ?)
      -> actually more complicated cf the supplementary material to the acl paper
  
  - chats are not annotated before the 1000th turn, then 
      - 500 turns for train and test
      - 250 for dev (-> hence better scores on dev)
      - what the fuck ? 
  - important features for turn-turn linking
      - same speaker (pair)
      - emoticon-only turn (often just a follow-up of previous turn) (turn)
      - speaker id mention (usually starting a turn "bob: blabla") is the same (pair)
      - speaker id mention in the turn-> generally indicate longer attachment (turn)
      - speaker id mention is same as speaker of other turn (pair)
      - time distance (pair)
      - nb of turn apart (pair)
      - presence of link -> should be preprocessed as a unique token (turn)
      - analysis de corrélation avec attachement pour sélection des traits
      - users ubotu/ubottu -> bots après commande/d'aide eg "!alsaconf"
      - tour commence par "!": generalement pour aider tour précédent, et automatiquement suivi de la réponse du bot ubuntu
  - to be checked: 
      - inconsistence of SELFLOOP / NO LINK ? cf guidelines
          - "If it appears to respond to something, but you can't find it, leave no link."
          - Interjections / Noise -> idem
      - nb of multiple heads. seems minor actually -> important to find best head
      - distribution of probs for best head candidate -> in relation to presence of link, to check correlation
      - check vocab / and embedding of OOV / what is done with them; would be sort of solved with BERT piece tokens

In [1]:
import itertools
import logging
from typing import Any, Tuple, Dict, List, Iterable
import torch
import torch.optim as optim

In [2]:
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.data.fields import Field, LabelField, TextField, ListField, SequenceLabelField
from allennlp.data.instance import Instance
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer, PretrainedBertIndexer
from allennlp.data.tokenizers import Token, Tokenizer, WordTokenizer, PretrainedTransformerTokenizer
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding, PretrainedBertEmbedder
from allennlp.modules.seq2vec_encoders import BertPooler

from allennlp.modules import Seq2VecEncoder, Seq2SeqEncoder
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.data.vocabulary import Vocabulary

from allennlp.training.trainer import Trainer
from allennlp.data.iterators import BucketIterator
from allennlp.common import Params
from allennlp.nn import util

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
#from dummy_chat_reader import ChatReader
from irc_chat_reader import ChatReader

In [5]:
BERT = False

In [6]:
token_indexers = {"tokens": SingleIdTokenIndexer()}

tokenizer_cfg = Params({"word_splitter": {"language": "en"}})

word_tokenizer = Tokenizer.from_params(tokenizer_cfg)

In [7]:
if BERT:
    bert_model = "bert-base-uncased"
    bert_cfg = {
        "tokenizer": {
                "type": "pretrained_transformer",
                "model_name": bert_model,
                "do_lowercase": True, 
                },
        "token_indexers": {
                        "type": "bert-pretrained",
                        "pretrained_model": bert_model,
                        "do_lowercase": True, 
                        "truncate_long_sequences":True,
                        "use_starting_offsets": True

            },
        "text_field_embedder": {
                "allow_unmatched_keys": True,
                "embedder_to_indexer_map": {
                "bert": ["bert", "bert-offsets"],
            },
        "token_embedders": {
                "bert": {
                    "type": "bert-pretrained",
                    "pretrained_model": bert_model 
                }
            }
        }
    }
    token_indexers = TokenIndexer.from_params(Params(bert_cfg["token_indexers"]))
    word_tokenizer = Tokenizer.from_params(Params(bert_cfg["tokenizer"]))
    #vocab ??? see below
    #word_embedding = TextFieldEmbedder.from_params(vocab,Params(bert_cfg["text_field_embedder"]))

In [8]:
if BERT:
    from types import SimpleNamespace
    bert_indexer = PretrainedBertIndexer(
            pretrained_model="bert-base-uncased",
            do_lowercase=True,
            truncate_long_sequences=True
        )
    token_indexers = {"bert":bert_indexer}
    # crucial piece since bert indexer acts as a tokenizer too
    def word_tokenizer_fun(s: str):
            return [Token(t) for t in bert_indexer.wordpiece_tokenizer(s)]
    # added to match irc_chat_reader expectations of a tokenizer
    word_tokenizer = SimpleNamespace(tokenize=word_tokenizer_fun)


    bert_embedder = PretrainedBertEmbedder(
            pretrained_model="bert-base-uncased",
            top_layer_only=False,
        )
    word_embedder: TextFieldEmbedder = BasicTextFieldEmbedder({"tokens": bert_embedder},
                                                                {"embedder_to_indexer_map": {
                                                                 "bert": ["bert", "bert-offsets"]},
                                                                "tokens": {"input_ids": "tokens",
                                                                              "offsets": "offset"}},
                                                                allow_unmatched_keys=True)
    

In [32]:
reader = ChatReader(
    tokenizer=word_tokenizer,
    token_indexers=token_indexers,
    raw = True,
    sub_sequence = None,
    #clip = 200
    )
train_instances = reader.read("../data/train")
dev_instances = reader.read("../data/dev")

153it [00:28,  5.28it/s]
10it [00:00, 11.72it/s]


In [33]:
vocab = Vocabulary.from_instances(train_instances+dev_instances)

100%|██████████| 163/163 [00:00<00:00, 313.93it/s]


In [34]:
#for i in train_instances:
#    #print(i)
#    i["lines"].index(vocab)
#    i["arcs"].index(vocab)
#    print(i["lines"].get_padding_lengths())

In [35]:
i0 = train_instances[0]
i0["is_server"].array[:10]

array([False, False, False, False, False, False, False, False, False,
       False])

In [36]:
#while True:
#    pass

In [37]:
#sum([len([x for x in i["metadata"].metadata["addressees"] if x is None]) for i in train_instances])

In [38]:
c = 0
for n,i in enumerate(train_instances):
    nbarcs = len(i["arcs"].indices)
    if nbarcs==0: 
        c+=1 
        print(n)
print("nb of document with zeros edges:",c)

nb of document with zeros edges: 0


In [39]:
#distance max
#for instance in train_instances:
#    maxl = max([abs(j-i) for (i,j) in instance["arcs"].indices])
#    print(maxl)
               

In [40]:
input_size = 50

if not BERT:
    turn_encoder_cfg = Params({"type":"gru",'input_size': input_size, 'hidden_size': 100, 'num_layers': 1,
                      'dropout': 0.25, 'bidirectional': True
})
    #can be changed dynamically encoder_cfg["type"] = "lstm"
    # warning: if bidirectional, state output dimension is hidden_size x 2 

    turn_encoder = Seq2VecEncoder.from_params(turn_encoder_cfg)
    turn_encoder.hidden_size = turn_encoder_cfg["hidden_size"]*(1+turn_encoder_cfg["bidirectional"])
else:
    turn_encoder = BertPooler("bert-base-uncased",requires_grad=True,dropout=0.0)
    turn_encoder.hidden_size = turn_encoder._embedding_dim

print(turn_encoder.hidden_size)


turn_feature_size = 1

chat_encoder_cfg = Params({"type":"gru",'input_size': turn_encoder.hidden_size, 'hidden_size': 100, 'num_layers': 3,
                  'dropout': 0.25, 'bidirectional': False
})
chat_encoder = Seq2SeqEncoder.from_params(chat_encoder_cfg)
chat_encoder.hidden_size = chat_encoder_cfg["hidden_size"]


if not BERT:
    #glove_dim, glove_version = 300, "https://s3-us-west-2.amazonaws.com/allennlp/datasets/glove/glove.6B.300d.txt.gz"
    glove_dim, glove_version = input_size, "../data/glove-ubuntu.txt"
    glove_text_field_embedder = Embedding.from_params(vocab,Params({"pretrained_file": glove_version,
                                                          "embedding_dim": glove_dim,
                                                          "trainable": True
    }))
    

    #token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
    #                        embedding_dim=300)
    token_embedding = glove_text_field_embedder 
    word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})

/home/muller/miniconda3/envs/allennlp2/lib/python3.7/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
30635it [00:00, 111245.59it/s]

200


61261it [00:00, 222682.06it/s]


In [41]:
if BERT: 
    word_embeddings = word_embedder


In [42]:
#chat_encoder

In [43]:
# from biaffine parser, another config (not used: assumes tokens)
chat_encoder_cfg =  Params({
            "type": "stacked_bidirectional_lstm",
            "hidden_size": 400,
            "input_size": 200,
            "num_layers": 3,
            "recurrent_dropout_probability": 0.3,
            "use_highway": True
        })


In [44]:
from allennlp.models import Model
from typing import Dict, List, Iterable
from allennlp.modules import TimeDistributed
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits

In [45]:
from allennlp.models import Model
from allennlp.common.checks import check_dimensions_match, ConfigurationError
from allennlp.data import Vocabulary
from allennlp.modules import Seq2SeqEncoder, TextFieldEmbedder, Embedding, InputVariationalDropout
from allennlp.modules.matrix_attention.bilinear_matrix_attention import BilinearMatrixAttention
from allennlp.modules.matrix_attention.dot_product_matrix_attention import DotProductMatrixAttention
from allennlp.modules.matrix_attention.linear_matrix_attention import LinearMatrixAttention

from allennlp.modules import FeedForward
from allennlp.models.model import Model
from allennlp.nn import InitializerApplicator, Activation
from allennlp.nn.activations import Activation
from allennlp.nn.util import add_positional_features


#???? TODO from allennlp.nn.util import min_value_of_dtype -> only allennlp >= 1.0
def min_value_of_dtype(dtype: torch.dtype):
    """
    Returns the minimum value of a given PyTorch data type. Does not allow torch.bool.
    """
    return info_value_of_dtype(dtype).min
def info_value_of_dtype(dtype: torch.dtype):
    """
    Returns the `finfo` or `iinfo` object of a given PyTorch data type. Does not allow torch.bool.
    """
    if dtype == torch.bool:
        raise TypeError("Does not support torch.bool")
    elif dtype.is_floating_point:
        return torch.finfo(dtype)
    else:
        return torch.iinfo(dtype)

# utility for max decoding. should be a better way but...
# should also not modify the original tensor ?
def force_max(t):
    """set all values of a tensor (batch,length,length) to zero if not maximum (on a given line)"""
    dims = t.size()
    b,_ = t.max(axis=2)
    bb = b.repeat((1,dims[1]))
    bb = torch.reshape(bb,(dims[0],dims[1],dims[1])).transpose(1,2)
    #print(bb)
    t[t<bb]=0
    return t
    
    

from allennlp.nn.util import get_text_field_mask
from allennlp.nn.util import get_lengths_from_binary_sequence_mask
from allennlp.training.metrics import F1Measure

import copy
from overrides import overrides
import torch
from torch.nn.modules import Dropout
from torch.nn.modules.linear import Linear
import numpy
import pandas as pds

class ChatGraphParser(Model):
    """
    A Parser for arbitrary graph structures.

    Registered as a `Model` with name "graph_parser".

    # Parameters

    vocab : `Vocabulary`, required
        A Vocabulary, required in order to compute sizes for input/output projections.
    text_field_embedder : `TextFieldEmbedder`, required
        Used to embed the `tokens` `TextField` we get as input to the model.
    turn_encoder : `Seq2VeqEncoder`
        The encoder that we will use to generate representation for whole turns from tokens.
    chat_encoder: `Seq2SeqEncoder`   The encoder that we will use to generate representations
        of turns within a chat
    arc_representation_dim : `int`, required.
        The dimension of the MLPs used for arc prediction.
        
    feature_size : `int`
        The embedding size for all the embedded features, such as distances
        
    tag_feedforward : `FeedForward`, optional, (default = None).
        The feedforward network used to produce tag representations.
        By default, a 1 layer feedforward network with an elu activation is used.
    arc_feedforward : `FeedForward`, optional, (default = None).
        The feedforward network used to produce arc representations.
        By default, a 1 layer feedforward network with an elu activation is used.

    dropout : `float`, optional, (default = 0.0)
        The variational dropout applied to the output of the encoder and MLP layers.
    input_dropout : `float`, optional, (default = 0.0)
        The dropout applied to the embedded text input.
    edge_prediction_threshold : `int`, optional (default = 0.5)
        The probability at which to consider a scored edge to be 'present'
        in the decoded graph. Must be between 0 and 1.
    initializer : `InitializerApplicator`, optional (default=`InitializerApplicator()`)
        Used to initialize the model parameters.
    """

    def __init__(
        self,
        vocab: Vocabulary,
        text_field_embedder: TextFieldEmbedder,
        turn_encoder: Seq2VecEncoder, 
        chat_encoder: Seq2SeqEncoder,
        arc_representation_dim: int,
        feature_size = 4,
        arc_feedforward: FeedForward = None,
        use_features = False,
        turn_feature_size = 0,# total dimension of turn-based additional features
        pair_feature_size = 0,# total dimension of pair-based additional features
        dropout: float = 0.3,
        input_dropout: float = 0.0,
        edge_prediction_threshold: float = 0.5,
        positive_class_weight = 40,
        prediction_window = 13, # dont predict edge further apart
        initializer: InitializerApplicator = InitializerApplicator(),
        debug = False,
        **kwargs,
    ) -> None:
        super().__init__(vocab, **kwargs)
        
        self.text_field_embedder = text_field_embedder
        self.turn_encoder = TimeDistributed(turn_encoder)
        self.chat_encoder = chat_encoder
        
        self.edge_prediction_threshold = edge_prediction_threshold
        if not 0 < edge_prediction_threshold < 1:
            raise ConfigurationError(
                f"edge_prediction_threshold must be between "
                f"0 and 1 (exclusive) but found {edge_prediction_threshold}."
            )

        encoder_dim = chat_encoder.get_output_dim()
        
        self.use_features = use_features
        self.turn_feature_size = turn_feature_size
        self.pair_feature_size = pair_feature_size
        
        self.head_arc_feedforward = FeedForward(
            encoder_dim+self.turn_feature_size, 1, arc_representation_dim, Activation.by_name("relu")()
        )
        self.child_arc_feedforward = copy.deepcopy(self.head_arc_feedforward)

        # 10 possible distance buckets.
        if self.use_features:
            self._num_distance_buckets = 10
            self._distance_embedding = Embedding(
                embedding_dim=feature_size, num_embeddings=self._num_distance_buckets
            )
            total_feature_size = self.pair_feature_size
        
        #
        #self.arc_attention = BilinearMatrixAttention(
        #    arc_representation_dim, arc_representation_dim, use_input_biases=True
        #)
        #self.arc_attention = DotProductMatrixAttention()
        self.arc_attention = LinearMatrixAttention(
            arc_representation_dim, arc_representation_dim, combination="x,y,x*y,x-y", activation = None
        )
           
        
        self._dropout = InputVariationalDropout(dropout)
        self._input_dropout = Dropout(input_dropout)
        # unused for now
        self.final_activation = Activation.by_name("sigmoid")
        # tested with distance feature (embedding) and spk_add boolean; final feedforward accounts also for attention scores
        if self.use_features:
            self.final_score = TimeDistributed(Linear(1+total_feature_size,1))#,self.final_activation))
        
        representation_dim = turn_encoder.get_output_dim()

        self._unlabelled_f1 = F1Measure(positive_label=1)
        #  with weight favoring recall of positive class 
        self._arc_loss = torch.nn.BCEWithLogitsLoss(reduction="none",
                                                    pos_weight=torch.tensor([positive_class_weight]))
        
        self.prediction_window = prediction_window
        initializer(self)
        # useful for debugging
        self.iter_count = 0 
        self.debug = debug
    # init done
        
    # todo 
    @overrides
    def forward(
        self,  # type: ignore
        lines,
        arcs: torch.LongTensor = None,
        rel_features = None,
        offsets = None,
        is_server = None,
        metadata: List[Dict[str, Any]] = None,
    ) -> Dict[str, torch.Tensor]:

        """
        # Parameters

        lines: the chat as a list of turns, each being a list of token
        TODO: add metadata to instances
        metadata : List[Dict[str, Any]], optional (default = None)
            A dictionary of metadata for each batch element which has keys:
                tokens : `List[str]`, required.
                    The original string tokens in the sentence.
        arcs : a tensor containing the adjacency matrix for the instance dependencies between turns
            Has shape `(batch_size, sequence_length, sequence_length)`.

        # Returns

        An output dictionary.
        """
        #########
        # this is the part where chat is encoded as sequence of turn encodings
        #########
        # mask for each turn of each chat of the batch: shape = (batch_size x max_turns x tokens)
        token_mask = get_text_field_mask(lines,num_wrapping_dims=1)

        # chat turns fetching embedding
        # turns_embedding tensor is (batch_size x turns x max tokens x token embedding size)
        turns_embeddings = self.text_field_embedder(lines,num_wrapping_dims=1)
      
        # encoding turns
        # turn_h has shape (batch_size x turns x encoder_output_size) 
        turn_h = self.turn_encoder(turns_embeddings,token_mask)
        
        # mask for chats is now nb of turns; beware weird return type of torch.max (tuple) 
        chat_mask = token_mask.max(axis=2)[0]
        
        # renaming to mask -> easier to transpose the rest of graph_parser
        mask = chat_mask
        
        # graph parser goes on
        # leave input dropout for now
        # embedded_text_input = turn_h equivalent in hierarchical sequence -> renaming 
        #embedded_text_input = self._input_dropout(embedded_text_input)
        embedded_text_input = turn_h
        
        # encoded_turns = encoded chat = self.chat_encoder(turn_h,chat_mask) equivalent in hierarchical sequence
        encoded_turns = self.chat_encoder(embedded_text_input, mask)
        #breakpoint()
        encoded_turns = self._dropout(encoded_turns)
        # experimentally, add positional sinusoidal encoding
        encoded_turns = add_positional_features(encoded_turns)
        
        if self.use_features:
            encoded_turns = torch.cat([encoded_turns,is_server.unsqueeze(-1)],dim=-1)
    
        # shape (batch_size, sequence_length, arc_representation_dim)
        head_arc_representation = self._dropout(self.head_arc_feedforward(encoded_turns))
        child_arc_representation = self._dropout(self.child_arc_feedforward(encoded_turns))

        # features for distance between candidate turns (unused for now)
        # should be refactored in a pair embedding dedicated function to integrate everything
        if self.use_features:
            distance_embeddings = self._distance_embedding(
                util.bucket_values(offsets, num_total_buckets=self._num_distance_buckets)
                )
        # todo: change to infersent composition
        #pair_embeddings = torch.cat(
        #    [
        #        target_embeddings,
        #        src_embeddings,
        #        src_embeddings * target_embeddings,
        #        distance_embeddings,
        #    ],
        #    -1,
        #)
        
        # Todo: change to simpler composition
        # shape (batch_size, sequence_length, sequence_length)
        arc_scores = self.arc_attention(head_arc_representation, child_arc_representation) 
        #breakpoint()
        # result here is size (batch,seq length,seq length, 1+total_feature_dim)
        if self.use_features: 
            final_rep = torch.cat([arc_scores.unsqueeze(-1),distance_embeddings,rel_features.unsqueeze(-1)],dim=-1)
            final_size = final_rep.size()
            # flattent seq lengthxlength to pass on to timedistributed
            final_rep_flat = final_rep.flatten(1,2)
            # should output batch x seqlenght*seqlength (x 1)
            #if self.debug:
            #    breakpoint()
            final_scores = self.final_score(final_rep_flat)
            # reshape to original matrix form batch x seqlength x seqlength
            final_scores = torch.reshape(final_scores,(final_size[0],final_size[1],final_size[2],1)).squeeze(-1)
            arc_scores = final_scores
        
        # shape (batch_size, num_tags, sequence_length, sequence_length)
        #arc_tag_logits = self.tag_bilinear(head_tag_representation, child_tag_representation)
        # Switch to (batch_size, sequence_length, sequence_length, num_tags)
        #arc_tag_logits = arc_tag_logits.permute(0, 2, 3, 1).contiguous()

        # Since we'll be doing some additions, using the min value will cause underflow
        # CHAT: unncessary since we dont have a loss for labels
        #minus_mask = ~mask * min_value_of_dtype(arc_scores.dtype) / 10
        #arc_scores = arc_scores + minus_mask.unsqueeze(2) + minus_mask.unsqueeze(1)

        self.iter_count += 1
        # Debugging every x batchs 
        if self.debug and self.iter_count%(10)==0:
            breakpoint()
            
        arc_probs = self._greedy_decode(arc_scores, mask,self.prediction_window)
        # heads: 
        # values , heads = arc_probs.max(dim=2)
        # should also include: threshold to have 
        # eg heads = heads[:,values>threshold] (beware of batch dimension!)
        # -> link to loss ? none ? 
        # but check attachement score for metric (needs tag though could be reproduced w/o them)
        
        output_dict = {"arc_probs": arc_probs, "mask": mask}

        if metadata:
            output_dict["metadata"] = metadata

        arc_tags = arcs # gold labels -> here just the adjacency matrix 0/1 ? 
        if arc_tags is not None:
            arc_nll= self._construct_loss(
                arc_scores=arc_scores, arc_tags=arc_tags, mask=mask
            )
            # same here with no arc relations ; keep all anyway to prevent ubgs downstream (TODO: coherent renaming)
            output_dict["loss"] = arc_nll 
            output_dict["arc_loss"] = arc_nll
            

            # Make the arc tags not have negative values anywhere
            # (by default, no edge is indicated with -1).
            # NB re chat: probably not useful, but kept as a precaution
            arc_indices = (arc_tags != -1).float()
            tag_mask = mask.unsqueeze(1) & mask.unsqueeze(2)
            one_minus_arc_probs = 1 - arc_probs
            # We stack scores here because the f1 measure expects a
            # distribution, rather than a single value.
            self._unlabelled_f1(
                torch.stack([one_minus_arc_probs, arc_probs], -1), arc_indices, tag_mask
            )
        
        
        return output_dict
    # modified / partially tested
    def _construct_loss(
        self,
        arc_scores: torch.Tensor,
        arc_tags: torch.Tensor,
        mask: torch.BoolTensor,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Computes the arc loss for an adjacency matrix.

        # Parameters

        arc_scores : `torch.Tensor`, required.
            A tensor of shape (batch_size, sequence_length, sequence_length) used to generate a
            binary classification decision for whether an edge is present between two words.
        arc_tags : `torch.Tensor`, required.
            A tensor of shape (batch_size, sequence_length, sequence_length).
            The labels for every arc (0/1).
        mask : `torch.BoolTensor`, required.
            A mask of shape (batch_size, sequence_length), denoting unpadded
            elements in the sequence.

        # Returns

        arc_nll : `torch.Tensor`, required.
            The negative log likelihood from the arc loss.
        tag_nll : `torch.Tensor`, required.
            The negative log likelihood from the arc tag loss.
        """
        arc_indices = (arc_tags != -1).float()
        # Make the arc tags not have negative values anywhere
        # (by default, no edge is indicated with -1).
        arc_tags = arc_tags * arc_indices
        arc_nll = self._arc_loss(arc_scores, arc_indices) * mask.unsqueeze(1) * mask.unsqueeze(2)
        
        # We want the mask for the tags to only include the unmasked words
        # and we only care about the loss with respect to the gold arcs.
        tag_mask = mask.unsqueeze(1) * mask.unsqueeze(2) * arc_indices

        #batch_size, sequence_length, _, num_tags = arc_tag_logits.size()
        #original_shape = [batch_size, sequence_length, sequence_length]
        #reshaped_logits = arc_tag_logits.view(-1, num_tags)
        reshaped_tags = arc_tags.view(-1)
        #tag_nll = (
        #    self._tag_loss(reshaped_logits, reshaped_tags.long()).view(original_shape) * tag_mask
        #)

        valid_positions = tag_mask.sum()

        arc_nll = arc_nll.sum() / valid_positions.float()
        #tag_nll = tag_nll.sum() / valid_positions.float()
        return arc_nll#, tag_nll
    
    
    # modified / untested
    # Warning: the initial Dozat & Manning implementation filter prediction with threshold here instead
    # of decoding, but not in latest 1.1 allennlp version
    #-------
    # this is supposed to be called when ? doc unclear. says with model.forward_on_instances but does not seem
    # to be the case (see tests below)
    #-------
    # no method for Model in used version (0.9?)
    #@overrides
    def make_output_human_readable(
        self, output_dict: Dict[str, torch.Tensor]
    ) -> Dict[str, torch.Tensor]:
        #breakpoint()
        arc_probs = output_dict["arc_probs"].cpu().detach().numpy()
        mask = output_dict["mask"]
        lengths = get_lengths_from_binary_sequence_mask(mask)
        arcs = []
        arc_tags = []
        for instance_arc_probs, length in zip(
            arc_probs, lengths
        ):

            arc_matrix = instance_arc_probs > self.edge_prediction_threshold
            edges = []
            #edge_tags = []
            for i in range(length):
                for j in range(length):
                    if arc_matrix[i, j] == 1:
                        edges.append((i, j))
                        #tag = instance_arc_tag_probs[i, j].argmax(-1)
                        #edge_tags.append(self.vocab.get_token_from_index(tag, "labels"))
            arcs.append(edges)
            #arc_tags.append(edge_tags)

        output_dict["arcs"] = arcs
        #output_dict["arc_tags"] = arc_tags
        return output_dict
    
    
    
    # modified/partially tested
    @staticmethod
    def _greedy_decode(
        arc_scores: torch.Tensor, 
        mask: torch.BoolTensor,
        prediction_window: int,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Decodes the head and head tag predictions by decoding the unlabeled arcs
        independently for each word and then again, predicting the head tags of
        these greedily chosen arcs independently.

        # Parameters

        arc_scores : `torch.Tensor`, required.
            A tensor of shape (batch_size, sequence_length, sequence_length) used to generate
            a distribution over attachments of a given word to all other words.
        ###arc_tag_logits : `torch.Tensor`, required.
        ###    A tensor of shape (batch_size, sequence_length, sequence_length, num_tags) used to
        ###    generate a distribution over tags for each arc.
        mask : `torch.BoolTensor`, required.
            A mask of shape (batch_size, sequence_length).

        # Returns

        arc_probs : `torch.Tensor`
            A tensor of shape (batch_size, sequence_length, sequence_length) representing the
            probability of an arc being present for this edge.
        ####arc_tag_probs : `torch.Tensor`
        ####    A tensor of shape (batch_size, sequence_length, sequence_length, sequence_length)
        ####    representing the distribution over edge tags for a given edge.
        """
        # Mask the diagonal, because we don't self edges.
        # WARNING: might not be the case for chats ? -> should be an option
        #inf_diagonal_mask = torch.diag(arc_scores.new(mask.size(1)).fill_(-float('inf')))
        
        # no edges going backwards
        triangle_upper_mask = torch.triu(arc_scores.new(mask.size(1),mask.size(1)).fill_(-float('inf')))
        # prevent edges between turns more than prediction_window turns appart
        up = 1-torch.triu(arc_scores.new(mask.size(1),mask.size(1)).fill_(1),diagonal=prediction_window)
        down = 1-torch.tril(arc_scores.new(mask.size(1),mask.size(1)).fill_(1),diagonal=-prediction_window)
        diag_mask = torch.log(((up+down)-1))
        # up_mask = torch.triu(arc_scores.new_zeros(mask.size(1),mask.size(1),diagonal=-15)
        # down_mask = torch.tril(arc_scores.new_zeros(mask.size(1),mask.size(1),diagonal=15)
        # away_mask = (up_mask == down_mask)
        #arc_scores = arc_scores + inf_diagonal_mask 
        arc_scores = arc_scores + diag_mask + triangle_upper_mask
        
        # shape (batch_size, sequence_length, sequence_length, num_tags)
        #arc_tag_logits = arc_tag_logits + inf_diagonal_mask.unsqueeze(0).unsqueeze(-1)
        # Mask padded tokens, because we only want to consider actual word -> word edges.
        # CHAT: this is the wrong torch version lol this does not work/ confusion int/bools
        # minus_mask = ~mask.unsqueeze(2)
        # CHAT: this should work with torch>1.4
        #minus_mask = (mask<1).unsqueeze(2)
        minus_mask = (mask.unsqueeze(1) & mask.unsqueeze(2))<1
        
        arc_scores.masked_fill_(minus_mask, -float('inf'))
        # 
        
        #arc_tag_logits.masked_fill_(minus_mask.unsqueeze(-1), -float('inf'))
        # shape (batch_size, sequence_length, sequence_length)
        arc_probs = arc_scores.sigmoid()
        # best arc option: set all but the best arc for a given target to 0 (== best head) 
        #force_max(arc_probs)
        
        # shape (batch_size, sequence_length, sequence_length, num_tags)
        #arc_tag_probs = torch.nn.functional.softmax(arc_tag_logits, dim=-1)
        return arc_probs#, arc_tag_probs
    # modified / untested
    
    @overrides
    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        metrics = {}
        precision, recall, f1_measure = self._unlabelled_f1.get_metric(reset)
        metrics["precision"] = precision
        metrics["recall"] = recall
        metrics["f1"] = f1_measure
        return metrics

    # since human readable not accessible, post process the outputs of
    # model.forward_on_instances
    # includes prob threshold here
    def extract_threshold(self,output_dict):
        instance_arc_probs = output_dict["arc_probs"]
        mask = output_dict["mask"]
        length = len(mask)
        if True:
            arc_matrix = instance_arc_probs > self.edge_prediction_threshold
            edges = []
            for i in range(length):
                for j in range(length):
                    if arc_matrix[i, j] == 1:
                        edges.append((i, j))

        output_dict["arcs"] = edges
        return output_dict

    def extract_best(self,output_dict):
        best_heads = output_dict["arc_probs"].argmax(axis=1)
        best_values = output_dict["arc_probs"].max(axis=1)
        heads = list(zip(range(len(best_heads)),zip(best_heads,best_values)))
        output_dict["arcs"] = heads
        return output_dict
    
    def predict_graph(self,instances,data_frame=False,best=False):
        """apply model on a bunch of instances

        data_frame: if True, produce a pandas DataFrame, otherwise just add explicit edges to model output

        best: if True, take only best head for each turn 
        """
        outputs = self.forward_on_instances(instances)
        
        if best:    
            outputs = [model.extract_best(x) for x in outputs]
        else:
            outputs = [model.extract_threshold(x) for x in outputs]
            
        if not data_frame: 
            return outputs

        # save only positive cases for now
        frame = []
        for one in outputs:
            source = one["metadata"]["file_source"]
            start_idx = one["metadata"]["first_line"]
            #print(source,start_idx)
            for (i,j) in one["arcs"]:
                if best: 
                    tid, label = j
                else:
                    tid, label = j,1
                frame.append((source,i+start_idx,tid+start_idx,"<sentence>","<sentence>",label))
        return pds.DataFrame(frame,columns=["source_file","source","target","sentence1","sentence2","label"])

In [46]:
arc_representation_dim = 50 
feature_size = 4

model = ChatGraphParser(vocab,word_embeddings,
                        turn_encoder,chat_encoder,arc_representation_dim,
                        use_features = True,
                        turn_feature_size = 1,
                        feature_size = feature_size,
                        pair_feature_size = 1+feature_size,
                        prediction_window = 10,
                        positive_class_weight = 1000,
                        debug=False,
                        edge_prediction_threshold=0.55)


if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1

#cuda_device = -1
    
    
# not used yet, but ready
from allennlp.training.optimizers import Optimizer 
trainer_cfg = Params({
        "cuda_device": cuda_device,
        "grad_norm": 5,
        "num_epochs": 100,
        "optimizer": {
            "type": "dense_sparse_adam",
            "betas": [
                0.9,
                0.9
            ]
        },
        "patience": 50,
})
opt_cfg = trainer_cfg.pop("optimizer")

In [49]:


#optimizer = optim.SGD(model.parameters(), lr=0.1)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# not tested
#optimizer = Optimizer.from_params(model_parameters=model.parameters(),
#                                  params=opt_cfg)

# "lines" <-> "tokens"
iterator = BucketIterator(batch_size=8,
                          sorting_keys=[("lines","list_num_tokens")])
iterator.index_with(vocab)


# requires Allennlp > 1.0
#from allennlp.training.trainer import EpochCallback
#class Epoch_nb(EpochCallback):
#    
#    def __call__(
#          self,
#          trainer: "GradientDescentTrainer",
#          metrics: Dict[str, Any],
#          epoch: int
#      ) -> None:
#        print("Epoch nb %d"%epoch,end="\t")

    

trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  num_epochs=20,
                  grad_norm=5,
                  patience=10,
                  cuda_device = cuda_device,
                  train_dataset=train_instances,
                  validation_dataset=dev_instances,
                  should_log_parameter_statistics = False
)

In [50]:
#%pdb off
trainer.train()

precision: 0.0717, recall: 0.6243, f1: 0.1286, loss: 946.3008 ||: 100%|██████████| 20/20 [00:14<00:00,  1.55it/s]
precision: 0.0834, recall: 0.7151, f1: 0.1494, loss: 659.8169 ||: 100%|██████████| 2/2 [00:00<00:00,  7.21it/s]
precision: 0.0718, recall: 0.6258, f1: 0.1287, loss: 923.8093 ||: 100%|██████████| 20/20 [00:14<00:00,  1.37it/s]
precision: 0.0834, recall: 0.7151, f1: 0.1494, loss: 644.6534 ||: 100%|██████████| 2/2 [00:00<00:00,  6.86it/s]
precision: 0.0718, recall: 0.6263, f1: 0.1288, loss: 922.8437 ||: 100%|██████████| 20/20 [00:14<00:00,  1.40it/s]
precision: 0.0834, recall: 0.7151, f1: 0.1494, loss: 650.4512 ||: 100%|██████████| 2/2 [00:00<00:00,  7.28it/s]
precision: 0.0718, recall: 0.6251, f1: 0.1287, loss: 930.9834 ||: 100%|██████████| 20/20 [00:14<00:00,  1.56it/s]
precision: 0.0834, recall: 0.7151, f1: 0.1494, loss: 637.6427 ||: 100%|██████████| 2/2 [00:00<00:00,  7.06it/s]
precision: 0.0718, recall: 0.6263, f1: 0.1289, loss: 913.4115 ||: 100%|██████████| 20/20 [00:14<

KeyboardInterrupt: 

In [26]:
trainer.iterator.__dict__

[autoreload of irc_chat_reader_modif failed: Traceback (most recent call last):
  File "/home/muller/miniconda3/envs/allennlp2/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 244, in check
    superreload(m, reload, self.old_objects)
  File "/home/muller/miniconda3/envs/allennlp2/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 376, in superreload
    module = reload(module)
  File "/home/muller/miniconda3/envs/allennlp2/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/muller/miniconda3/envs/allennlp2/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/muller/Devel/Linto/linto-sp5-conversational-thread-extraction/experiments/irc_chat_reader_m

{'vocab': Vocabulary with namespaces:  tokens, Size: 45682 || Non Padded Namespaces: {'*tags', '*labels'},
 '_batch_size': 32,
 '_max_instances_in_memory': None,
 '_instances_per_epoch': None,
 '_maximum_samples_per_batch': None,
 '_cache_instances': False,
 '_cache': defaultdict(list, {}),
 '_track_epoch': False,
 '_epochs': defaultdict(int, {140455247675056: 11, 140454605160928: 11}),
 '_cursors': {},
 '_sorting_keys': [('lines', 'list_num_tokens')],
 '_padding_noise': 0.1,
 '_biggest_batch_first': False,
 '_skip_smaller_batches': False}

In [29]:
dev_instances = reader.read("../data/dev")

2497it [00:00, 2609.87it/s]


In [28]:
outputs = model.forward_on_instances(dev_instances)
output_dict = outputs[0]
best_heads = output_dict["arc_probs"].argmax(axis=1)
best_values = output_dict["arc_probs"].max(axis=1)
heads = list(zip(range(len(best_heads)),zip(best_heads,best_values)))

RuntimeError: CUDA out of memory. Tried to allocate 2.92 GiB (GPU 0; 10.73 GiB total capacity; 3.17 GiB already allocated; 1.08 GiB free; 3.19 GiB reserved in total by PyTorch)

In [ ]:
outputs

In [ ]:
outputs = model.predict_graph(dev_instances,best=True,data_frame=True)

In [ ]:
outputs

In [ ]:
# probing the gold annotations to see how many multiple attachements
gold = {}
for (i,j) in dev_instances[1]["arcs"].indices:
    if i in gold:
        gold[i].append(j)
    else:
        gold[i] = [j]

In [ ]:
from collections import Counter
c = Counter([len(gold[i]) for i in gold])
c

In [ ]:
start=0
nb = 200
turn_key = "lines"
for i,turn in enumerate(dev_instances[0][turn_key][start:start+nb]):
    k = i + start
    head = gold.get(k,"NONE")
    if head == [k]: head = "SELF-LOOP"
    print(k,"->",head,turn.tokens)

In [ ]:
sorted(dev_instances[0]["arcs"].indices)

In [ ]:
# (from tutorial) Here's how to save the model.
with open("/tmp/model.th", 'wb') as f:
    torch.save(model.state_dict(), f)
vocab.save_to_files("/tmp/vocabulary")
# And here's how to reload the model.
vocab2 = Vocabulary.from_files("/tmp/vocabulary")
model2 = ChatGraphParser(vocab2,word_embeddings,
                        turn_encoder,chat_encoder,arc_representation_dim,
                        prediction_window = 10,
                        positive_class_weight = 1000,
                        debug=False,
                        edge_prediction_threshold=0.6)
with open("/tmp/model.th", 'rb') as f:
    model2.load_state_dict(torch.load(f))
if cuda_device > -1:
    model2.cuda(cuda_device)

In [ ]:
#model2.state_dict()